To do:

- Read demand data
- Mean demand --> discrete requests (random issue+pick-up time)
    - Is there any demand in this running instance for this od-pair?
    - If there is demand > 0 for an od, then put it at random points during the peak hour (=attach a pick-up time between 0 and 60)
- Import OD-matrix (travel times)

In [4]:
import pandas as pd
import numpy as np

In [5]:
#lambdapeak: Mean demand (arrivals/min.) to travel towards the city center during peak hours
#mupeak: Mean demand (arrival/min.) to travel towards the terminal during peak hours

lambdapeak = pd.read_excel('Data/Demand rates/DemandInputSmall3.xlsx',sheet_name=0,header=0, index_col=(0,1), dtype=float)
mupeak = pd.read_excel('Data/Demand rates/DemandInputSmall3.xlsx',sheet_name=1,header=0, index_col=(0,1), dtype=float)

In [6]:
def load_demand_scenario(lambdapeak, mupeak, direction=None, scenario=1): #perhaps add another parameter for small/large/...
    if direction == 'city':
        result = lambdapeak.to_dict()
    elif direction == 'terminal':
        result = mupeak.to_dict()
    
    return result[scenario]

lambdapeak_scen = load_demand_scenario(lambdapeak, mupeak, 'city', 3)

In [7]:
#calculate whether for this loading instance there are any requests per od-pair

lambdapeak_scen

{(1, 1): 4.0,
 (1, 2): 4.0,
 (1, 3): 0.0,
 (2, 1): 4.0,
 (2, 2): 4.0,
 (2, 3): 0.0,
 (3, 1): 4.0,
 (3, 2): 4.0,
 (3, 3): 0.0}

In [8]:
# this function generates the discrete requests 
# Q: I've made it now so that every OD-pair where there is a mean demand > 0 has at least 1 request
# I now have a lot of requests close to each other, at which point can I aggregate?

peak_hour_duration = 60

def generate_requests(mean_demand):

    requests = {}

    for od in mean_demand.keys():
        if mean_demand[od] > 0:
    
            requests_od = []
            requests[od] = requests_od
            requests[od].append(0)

            t=0

            while t < peak_hour_duration:
                delta_t = np.random.exponential(1/mean_demand[od], 1)[0]  #the time before the next passenger arrives is sampled from an exponential distr.                              
                t += delta_t
            
                if t > 60:
                    break
                else:
                    requests[od].append(t) #if no request yet is at that time

    return requests

lambdapeak_requests = generate_requests(lambdapeak_scen)
lambdapeak_requests


{(1, 1): [0,
  0.11145408143019157,
  0.9755676401261315,
  1.0503932523054267,
  1.2875488217540731,
  1.831794869664268,
  2.353528137671813,
  2.41146022316471,
  2.5117460382424923,
  2.520677036612037,
  2.533408232477318,
  2.700944666193735,
  2.887368366093654,
  2.901306268792218,
  2.9394812032561766,
  3.18478801793602,
  3.2348517635947522,
  3.5695826404043594,
  3.5847264651094513,
  3.8571037912727144,
  4.161039868521737,
  4.252404367173132,
  4.603549518739241,
  4.8341707954107385,
  6.155153630618335,
  6.29775894106828,
  6.329062548751932,
  7.026985121704771,
  7.75791006604838,
  7.929129907084156,
  7.9847911343544,
  8.17803420750786,
  8.337422393673656,
  8.829095307391567,
  8.953202748317267,
  8.997799658148011,
  9.024410194095147,
  9.561774311976382,
  9.904745881873497,
  10.242342696938778,
  10.28593488429245,
  10.324323423768803,
  10.384045219384745,
  10.444769599940281,
  10.519097300502297,
  10.568764774140371,
  11.284748418287634,
  12.0707

In [12]:
# create individual requests. R = ((o,d), pickuptime)

lst_of_ind_requests = []

for k in lambdapeak_requests.keys():
    for v in lambdapeak_requests[k]:
        lst_of_ind_requests.append((k, v))
        
len(lst_of_ind_requests)

1429